In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pytickersymbols import PyTickerSymbols

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
#This function calculates the RSI
def RSI(data, window=14):
    #daily_data = data.resample('D').last()
    
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

#The function that calculates the strategies returns for each individual stock
def strat(data, ticker, buy, sell):     
    # Calculate RSI and add it to the DataFrame
    data['RSI'] = RSI(data)
    
    # Define the buy and sell conditions based on RSI
    data['Signal'] = 0
    data['Signal'][data['RSI'] > buy] = 1   # Buy signal
    data['Signal'][data['RSI'] > sell] = -1  # Sell signal
    
    # Shift signals so we enter positions on the next day
    data['Position'] = data['Signal'].shift(1)
    
    
    # Calculate daily returns
    data['Daily Return'] = data['Close'].pct_change()
    # Calculate strategy returns: Position * Daily Return
    data[ticker] = data['Position'] * data['Daily Return']
    
    return data[ticker]

In [ ]:
def calculate_portfolio_sharpe_ratio(daily, risk_free_rate=0.045):
    daily = daily.drop(daily.index[0]) #drops the inital row 

    #Removes any stocks that are missing data
    if daily.isnull().values.any():
        #print(f"Warning: Missing data for some stocks in the portfolio.")
        null_counts = daily.isnull().sum()  # Get the number of nulls per column
        non_zero_nulls = null_counts[null_counts > 0]  # Filter out the columns where null count is 0
        #print(non_zero_nulls)  Print only columns with null values
        daily = daily.dropna(axis=1, how='any')  # Drop stocks with missing data

    num = daily.shape[1] #Number of stocks in the portfolio
    weight = [1 / num] * num #An array of equal weighting of stocks
    weights = np.array(weight) #Converts weight to a numpy array
    
    portfolio_daily_returns = daily.dot(weights) #Calculates the daily strategy returns of the whole portfolio
    
    # Calculate annualized return for the portfolio
    annualized_return = ((1 + portfolio_daily_returns.mean()) ** 252) - 1

    # Calculate annualized volatility (standard deviation) for the portfolio
    annualized_volatility = portfolio_daily_returns.std() * np.sqrt(252)
    
    # Sharpe ratio formula: (Expected Return - Risk-Free Rate) / Volatility
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
    
    return sharpe_ratio

In [ ]:
#Creating an empty df to hold the results
parameters = pd.DataFrame({'Parameters':[], 'Sharpe':[]})
parameters.set_index('Parameters', inplace=True)


In [ ]:
#Importing the s&p500 stock list
stock_data = PyTickerSymbols()
sp500_symbols = stock_data.get_stocks_by_index('S&P 500')
symbols = [stock['symbol'] for stock in sp500_symbols]
#symbols = ["AAPL", "NVDA", "MSTR"] #Test data

pulled_data = yf.download(symbols, period = "1y", interval = "1d", group_by="ticker") #Pulling all the data for all the stocks

#Loops through each parameters going from buy(50-100), sell(70-120), keeping a 20 spread
sell = 60
for buy in range(40,101):
    #Creating an empty dataframe that will hold the daily strategy returns of each stock 
    daily = pd.DataFrame({'Date': []})
    daily.set_index('Date', inplace=True)

    #Loops through each stock in the s&p500
    for symbol in symbols:
        #Uses the functions above to calculate the strat daily returns for the stock and stores it in the daily df
        individual_stock_data = pulled_data[symbol]
        strategy_returns = strat(individual_stock_data, symbol, buy, sell)
        daily = pd.concat([daily, strategy_returns], axis=1)

    #Calculates the sharpe ratio for the portfolio and stores it in the parameters df
    sharpe = calculate_portfolio_sharpe_ratio(daily)
    pars = str(buy) + "," + str(sell)
    new_row = pd.DataFrame({'Sharpe': [sharpe]}, index=[pars])
    parameters = pd.concat([parameters, new_row])
    
    sell += 1

In [ ]:
parameters

In [ ]:
parameters.to_csv('40 start 1y.csv', index=True)